Описание
Исходный репозиторий: https://github.com/lukalabs/cakechat

Логика работы:



Предварительные действия
* нужно установить зависимости питона в первых двух ячейках, после этого перезагрузить рантайм(!)
* репозиторий клонируется в виртуальную машину колаба
* при первом запуске кода, он автоматически выкачивает модель из s3 хранлищиа, в конце ноутбука есть ячейка копирующая эти файлы в гугл диск. Их можно скопировать и распаковать `/content/cakechat/results/nn_models`
* к сожалению обучающий датасет отсутствует. обучение велость на закрытом датасете твиттера. По ссылке есть формат датасета для обучения https://github.com/lukalabs/cakechat/blob/master/data/corpora_processed/ train_processed_dialogs.txt и ссылка где можно получить датасеты (но в другом формате) https://breakend.github.io/DialogDatasets/ их нужно готовить 
* Для разметки эмоций в своем датасете предлагается использовать https://github.com/bfelbo/DeepMoji
* Авторы рекомендуют использовать трансформационные диалоговые модели (GPT) как более песпективные, например рекомендуют https://github.com/microsoft/DialoGPT

Особенности модели:
* модель обучена на текстах до 30 токенов, большие отброшены
* глубина контекста - три сообщения, т.е. предыдущий вопрос, ответ и текущей текст
* система построена по модели декодирования и последующего кодирования
* декодер может учитывать различые категории, например в данном случае эмоции

Логика работы диалоговой системы:
* система реализована как веб-сервис
* в ноутбуке вебсервис распакован в опросную форму, с сохранением истории диалога.
* веб-сервис может быть подключен к телеграм боту.
* сама логика диалога приведена в реадми к репозиторию.



In [ ]:
!pip install tensorflow==1.13.1 tensorboard==1.13.0

# НЕОБХОДИМА ПЕРЕЗАГРУЗКА RUNTIME!!!!!!!

     |████████████████████████████████| 92.6MB 104kB/s 
     |████████████████████████████████| 3.2MB 39.7MB/s 
     |████████████████████████████████| 368kB 59.8MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.13.1 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:


In [ ]:
!pip install numpy==1.17.0 pandas==1.1.0 unicodecsv==0.14.1 requests==2.23.0 nltk==3.4.5 scipy==1.2.0 scikit-learn==0.20.2 
!pip install keras==2.2.4 telepot==12.6 gensim==1.0.1 cachetools==3.0.0 gunicorn==19.9.0 tqdm==4.30.0
!pip install boto3
# НЕОБХОДИМА ПЕРЕЗАГРУЗКА RUNTIME!!!!!!!

In [2]:
# Репозиторий храним в виртуалке ноутбука
import os
import sys
CAKECHAT_PATH='/content/cakechat'
if not os.path.exists(CAKECHAT_PATH):
  !git clone https://github.com/lukalabs/cakechat.git $CAKECHAT_PATH

%cd $CAKECHAT_PATH
%ls

Cloning into '/content/cakechat'...
remote: Enumerating objects: 437, done.
remote: Total 437 (delta 0), reused 0 (delta 0), pack-reused 437
Receiving objects: 100% (437/437), 597.60 KiB | 3.34 MiB/s, done.
Resolving deltas: 100% (160/160), done.
/content/cakechat
bin/       data/         LICENSE    requirements-local.txt  tools/
cakechat/  dockerfiles/  README.md  requirements.txt


In [4]:
# Инициализация, источник https://github.com/lukalabs/cakechat/tree/master/bin


#sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
if CAKECHAT_PATH not in sys.path:  # Добавлениеп библиотеки cakechat в пути для поиска
    sys.path.append(CAKECHAT_PATH)
from cakechat.utils.env import set_keras_tf_session

gpu_memory_fraction = os.environ.get('GPU_MEMORY_FRACTION', 0.1)
set_keras_tf_session(gpu_memory_fraction)

# from cakechat.api.v1.server import app # Вместо подключения фласк приложения, реализуем логику коносли



In [6]:
# from cakechat/api/v1/server.py
from cakechat.api.response import get_response
from cakechat.api.utils import get_api_error_response, parse_dataset_param
from cakechat.config import EMOTIONS_TYPES, DEFAULT_CONDITION
from cakechat.utils.logger import get_logger

import json

_logger = get_logger(__name__)

# @app.route('/cakechat_api/v1/actions/get_response', methods=['POST'])
# def get_model_response():
def get_model_response(params):
#params = request.get_json()
# params = {
#  'context': ['Hello', 'Hi!', 'How are you?'],
#  'emotion': 'joy'
# }
  _logger.info('request params: {}'.format(params))

  try:
      dialog_context = parse_dataset_param(params, param_name='context')
  except KeyError as e:
      return get_api_error_response('Malformed request, no "{}" param was found'.format(e), 400, _logger)
      # print(get_api_error_response('Malformed request, no "{}" param was found'.format(e), 400, _logger))
  except ValueError as e:
      return get_api_error_response('Malformed request: {}'.format(e), 400, _logger)
      # print(get_api_error_response('Malformed request: {}'.format(e), 400, _logger))

  emotion = params.get('emotion', DEFAULT_CONDITION)
  if emotion not in EMOTIONS_TYPES:
      return get_api_error_response(
          'Malformed request, emotion param "{}" is not in emotion list {}'.format(emotion, list(EMOTIONS_TYPES)),
          400, _logger)
  #     print(return get_api_error_response(
  #         'Malformed request, emotion param "{}" is not in emotion list {}'.format(emotion, list(EMOTIONS_TYPES)),
  #         400, _logger)
  # )

  response = get_response(dialog_context, emotion)
  _logger.info('Given response: "{}" for context: {}; emotion "{}"'.format(response, dialog_context, emotion))

  # return jsonify({'response': response}), 200
  return json.dumps({'response': response})

# @app.errorhandler(Exception)
# def on_exception(exception):
#     return get_api_error_response('Can\'t process request: {}'.format(exception), 500, _logger)

ModuleNotFoundError: ignored

In [ ]:
# Проверка работы
res = get_model_response({
 'context': ['Hello', 'Hi!', 'How are you?'],
 'emotion': 'joy'
})
print(res)

[14.05.2021 06:23:14.394][INFO][995][cakechat.utils.profile][14] Elapsed time for "rerank_candidates": 3.2529006004333496


{"response": "I'm good thanks. You?"}


In [ ]:
# Инициализация хранения истории диалога
message_history = []

In [ ]:
#@title Default title text
# Текст вопроса, в том числе предыдущих три
# Тип реакции: 'neutral', 'anger', 'joy', 'fear', 'sadness'
message = "What?" #@param {type:"string"}
emotion = "joy" #@param {type:"string"}

message_history.append(message)

contex = message_history[-3:]
res = get_model_response({
 'context': contex,
 'emotion': emotion
})
print('Ответ', res)
res_data=json.loads(res)

# dialog_history.append(res_data['response'])
message_history.append(res_data['response'])
for i, msg in enumerate(message_history):
  if i % 2 == 0:
    print(f'- {msg}')
  else:
    print(f'{msg}')

[14.05.2021 07:16:28.972][INFO][995][cakechat.utils.profile][14] Elapsed time for "rerank_candidates": 3.4175281524658203


Ответ {"response": "Your just so cute"}
- Hi
Hey!!! You look pretty in every picture of a girl
- What?
Your just so cute


In [ ]:
# Скопировать модели в хранилище
!cp /content/cakechat/results/nn_models/cakechat_v2.0_keras_tf_1bced8e69869.tar.gz $BASE_PATH/cakechat_v2.0_keras_tf_1bced8e69869.tar.gz
!cp /content/cakechat/results/nn_models/reverse_cakechat_v2.0_keras_tf_6648fddd592a.tar.gz $BASE_PATH/reverse_cakechat_v2.0_keras_tf_6648fddd592a.tar.gz